In [ ]:
"""
BSM functions (page 52 )
"""

In [ ]:
from math import log,sqrt,exp
from scipy import stats
import pandas as pd

In [ ]:
"""
Valuation of European call options in BSM model including 
Vega function and implied volatility estimation
"""

In [ ]:
#Analytical BSM formula

def bsm_call_value(S0,K,T,r,sigma):
    """
    parameters :
    S0 : float ; initial stock / index level
    K  : float ; strike price
    T  : float ; maturity date(in year fractions)
    r  : float ; constant risk free short rate
    sigma : float ; volatility factor in diffusion term
    
    
    Return :
    value : float ; present value of the European call option
    """
    
    S0 = float(S0)
    d1 = (log(S0 / K) + (r + 0.5 * sigma**2)*T )/ (sigma * sqrt(T))
    d2 = (log(S0 / K) + (r - 0.5 * sigma**2)*T )/ (sigma * sqrt(T))
    
    value = (S0 * stats.norm.cdf(d1,0.0,1.0) - K*exp(-r*T) * stats.norm.cdf(d2,0.0,1.0))
    
    return value

In [ ]:
# Vega function
def bsm_vega(S0,K,T,r,sigma):
    """
    parameters :
    S0 : float ; initial stock / index level
    K  : float ; strike price
    T  : float ; maturity date(in year fractions)
    r  : float ; constant risk free short rate
    sigma : float ; volatility factor in diffusion term
    
    Return :
    vega : float ; partial derivative of BSM formula with respect to sigma
    """
    S0 = float(S0)
    d1 = (log(S0 / K) + (r + 0.5 * sigma**2)*T )/ (sigma * sqrt(T))
    vega = S0*stats.norm.cdf(d1,0.0,1.0)*sqrt(T)
    return vega

In [ ]:
# implied volatility function
def bsm_call_imp_vol(S0,K,T,r,C0,sigma_est,it=100):
    """
    parameters :
    S0 : float ; initial stock / index level
    K  : float ; strike price
    T  : float ; maturity date(in year fractions)
    r  : float ; constant risk free short rate
    sigma_est : float ; estimate of implied volatility
    it : integer ; number of iterations
    
    
    Return :
    sigma_est : float ; numerically estimated implied volatility
    """
    
    for i in range(it):
        sigma_est -=((bsm_call_value(S0,K,T,r,sigma_est) - C0) / bsm_vega(S0,K,T,r,sigma_est))
        
    return sigma_est

In [ ]:
# values as per book
# data as uploaded by author
# reference date taken : March 31 , 2014

V0 = 17.6639
r = 0.01

h5 = pd.HDFStore('./home/cassandra/Shikha/Study/fin/Codes/vstoxx_data_31032014.h5','r')
futures_data = h5['futures_data']
options_data = h5['options_data']  # VSTOXX call option data
h5.close()


In [ ]:
futures_data

In [ ]:
options_data.info()

In [ ]:
options_data[['DATE','MATURITY','TTM','STRIKE','PRICE']].head()

In [ ]:
options_data['IMP_VOL'] = 0.0  # new column for implied volatilities

tol = 0.5 # tolerance level for moneyness

for option in options_data.index :
    #iterating over all option quotes
    forward = futures_data[futures_data['MATURITY'] == options_data.loc[option]['MATURITY']]['PRICE'].values[0]
    
    #picking the right fututes value
    if(forward*(1-tol) < options_data.loc[option]['STRIKE'] < forward * (1+tol)):
        #only for options with moneyness within tolerance
        imp_vol = bsm_call_imp_vol(V0,options_data.loc[option]['STRIKE'],options_data.loc[option]['TTM'],r,options_data.loc[option]['PRICE'],sigma_est = 2.,it=100)
        
        options_data['IMP_VOL'].loc[option] = imp_vol
        
        

In [ ]:
plot_data = options_data[options_data['IMP_VOL']>0]
maturities = sorted(set(options_data['MATURITY']))
maturities

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(8,6))
for maturity in maturities:
    data = plot_data[options_data.MATURITY == maturity]
    plt.plot(data['STRIKE'],data['IMP_VOL'], label = maturity.date() , lw = 1.5)
    plt.plot(data['STRIKE'],data['IMP_VOL'],'r.')
plt.grid(True)
plt.xlabel('strike')
plt.ylabel('implied volatility of volatility')
plt.legend()
plt.show()


In [ ]:
#group by maturity, then strike and keep the price and impl vol columns

keep = ['PRICE','IMP_VOL']
group_data = plot_data.groupby(['MATURITY','STRIKE'])[keep] # DataFrameGroupBy object is returned
group_data

In [ ]:
# ?? to egt the data we need to apply an aggregation op on the object eg sum. taking the sum yields the single data point since there is only one data element in every group
group_data = group_data.sum()
group_data.head()

In [ ]:
# all values that the indices (maturity and strike) can take
group_data.index.levels